In [275]:
import numpy as np
from collections import Counter, OrderedDict
from math import floor,ceil
from tqdm.notebook import tqdm

In [8]:
def maxelements(seq):
    ''' Return list of position(s) of largest element '''
    max_indices = []
    if seq:
        max_val = seq[0]
        for i,val in ((i,val) for i,val in enumerate(seq) if val >= max_val):
            if val == max_val:
                max_indices.append(i)
            else:
                max_val = val
                max_indices = [i]

    return max_indices

In [83]:
def avail_color(color_list):
    i=0
    while True:
        if i not in color_list:
            return i
        i += 1
        
def greedy(neigh_nds,nd_order):
    color = dict()
    for nd in nd_order:
        used_neighbour_colors = [color[nbr] for nbr in neigh_nds[nd]
                                 if nbr in color]
        color[nd] = avail_color(used_neighbour_colors)
    return color

In [194]:
def color_sort(sol):
    chr_nbr=max(sol)+1
    #sort by color and degrees
    order=[]
    for i in range(chr_nbr):
        c_list=[]
        for j in range(len(sol)):
            if sol[j]==i:
                c_list.append(j)
        order.append(c_list)
    return order

In [270]:
def it_greedy_solver(input_data,maxit):
    # parse the input
    lines = input_data.split('\n')

    first_line = lines[0].split()
    node_count = int(first_line[0])
    edge_count = int(first_line[1])

    edges = []
    for i in range(1, edge_count + 1):
        line = lines[i]
        parts = line.split()
        edges.append((int(parts[0]), int(parts[1])))

    #defines list of neighbor nodes
    neigh_nds=[]
    for i in range(node_count):
        neigh_nds_t=[]
        for k in range(edge_count):
            if (i in edges[k]):
                idx=edges[k].index(i)
                neigh_nds_t.append(edges[k][(idx-1)])
        neigh_nds.append(neigh_nds_t)   
    
    nd_lgth=[]
    for n in neigh_nds:
        nd_lgth.append(len(n))
    
    
    #nodes sorted by number of neighbors: (nd, #of nghbrs)
    sorted_nds=sorted(list(enumerate(nd_lgth)),key=lambda x:x[1],reverse = True)
    #print('sorted_nds:',sorted_nds)
    
    #gets the actual node numbers from sorted list
    nd_order = []
    for t in sorted_nds:
        nd_order.append(t[0])
    #print('nd_order:',nd_order)
    
    solution=greedy(neigh_nds,nd_order)
    solution=OrderedDict(sorted(solution.items()))
    sol=list(solution.values())
    #print('color_sort:',color_sort(sol))
    #loop greedy and sorting
    for i in tqdm(range(maxit)):
        #group nodes by color
        chr_sort_sol=color_sort(sol)
        
        #sort nodes by nbr of neighbors
        chr_deg_sol_list=[]
        for i in chr_sort_sol:
            chr_deg_sol=sorted(i,key=lambda x:nd_lgth[x],reverse=True)
            chr_deg_sol_list.append(chr_deg_sol)
        #print('color order sort:',chr_deg_sol_list) 
        
        #make random permutation of colors and extend list
        nd_order=[]
        rand_perm=np.random.permutation(len(chr_deg_sol_list))
        for i in rand_perm:
            nd_order.extend(chr_deg_sol_list[i])     
        #print('new nd_order:',nd_order)
        
        #find new solution
        solution=greedy(neigh_nds,nd_order)
        solution=OrderedDict(sorted(solution.items()))
        sol=list(solution.values())
        #print('')
        #end loop
        
    #chrom_nbr=print(max(solution.values()))   
    output_data = str(max(sol)+1) + ' ' + str(0) + '\n'
    output_data += ' '.join(map(str, sol))
    return output_data

In [276]:
import sys  
import os
notebook_path = os.path.abspath("Notebook.ipynb")
data_path = os.path.join(os.path.dirname(notebook_path), "data\\gc_500_3")

if len(data_path) > 1:
    file_location = data_path.strip()
    with open(file_location, 'r') as input_data_file:
        input_data = input_data_file.read()
    print(it_greedy_solver(input_data,10000))


38 0
5 11 18 15 6 8 11 0 21 32 2 24 10 4 32 0 12 25 23 22 21 1 35 30 11 18 34 9 30 20 21 31 24 32 14 30 36 20 18 35 34 7 13 10 1 12 9 17 7 12 24 29 22 17 3 9 37 3 26 2 28 11 34 28 22 24 21 0 0 12 22 3 7 3 7 24 27 17 17 23 30 12 35 9 12 5 28 27 22 1 19 27 32 15 15 12 17 15 36 27 10 5 3 1 22 20 5 15 25 7 7 13 9 5 18 10 34 23 31 5 5 7 36 0 29 13 18 20 6 18 21 2 12 12 7 14 23 1 2 28 16 23 2 18 32 30 33 10 19 6 32 29 18 27 10 5 21 20 33 1 4 5 18 0 37 1 17 36 10 22 4 14 31 36 33 35 8 0 23 3 12 18 5 23 9 31 26 29 0 21 13 7 0 3 3 11 10 2 0 31 11 17 35 9 35 2 8 27 24 15 31 35 23 10 24 6 16 11 29 1 26 16 15 34 36 30 24 26 3 7 8 18 0 13 12 25 24 13 31 20 8 28 20 17 9 7 27 34 8 31 15 25 35 17 19 6 12 19 33 5 8 10 36 21 6 30 31 20 33 29 21 21 32 27 25 3 13 1 4 33 23 6 16 11 24 25 18 28 20 8 23 19 6 28 1 19 35 9 34 5 34 37 6 14 21 9 29 5 14 16 4 28 19 36 9 14 9 4 17 1 3 24 28 2 34 21 4 25 14 3 36 26 11 15 8 30 7 34 26 12 11 12 12 6 0 25 17 34 37 20 15 14 29 19 8 2 6 4 0 1 4 4 11 2 36 2 7 4 10 37 26